<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Optuna-Tutorial" data-toc-modified-id="Optuna-Tutorial-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optuna Tutorial</a></span><ul class="toc-item"><li><span><a href="#The-theory-behind-Hyperparameter-tuning" data-toc-modified-id="The-theory-behind-Hyperparameter-tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>The theory behind Hyperparameter tuning</a></span><ul class="toc-item"><li><span><a href="#Notes-from-Video" data-toc-modified-id="Notes-from-Video-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Notes from Video</a></span></li><li><span><a href="#&quot;N-vs-B/N&quot;-Tradeoff" data-toc-modified-id="&quot;N-vs-B/N&quot;-Tradeoff-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>"N vs B/N" Tradeoff</a></span></li><li><span><a href="#Samplers-(Search-algo)" data-toc-modified-id="Samplers-(Search-algo)-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Samplers (Search algo)</a></span></li><li><span><a href="#Schedulers-(Prunner)" data-toc-modified-id="Schedulers-(Prunner)-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Schedulers (Prunner)</a></span></li><li><span><a href="#Optuna-demo---HP-Tuning-Steps" data-toc-modified-id="Optuna-demo---HP-Tuning-Steps-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Optuna demo - HP Tuning Steps</a></span><ul class="toc-item"><li><span><a href="#Search-space" data-toc-modified-id="Search-space-2.1.5.1"><span class="toc-item-num">2.1.5.1&nbsp;&nbsp;</span><b>Search space</b></a></span></li><li><span><a href="#RL-Objective-Function" data-toc-modified-id="RL-Objective-Function-2.1.5.2"><span class="toc-item-num">2.1.5.2&nbsp;&nbsp;</span><b>RL Objective Function</b></a></span></li><li><span><a href="#Choose-sampler,-pruner-and-launch-the-study" data-toc-modified-id="Choose-sampler,-pruner-and-launch-the-study-2.1.5.3"><span class="toc-item-num">2.1.5.3&nbsp;&nbsp;</span><b>Choose sampler, pruner and launch the study</b></a></span></li></ul></li><li><span><a href="#Common-pitfalls" data-toc-modified-id="Common-pitfalls-2.1.6"><span class="toc-item-num">2.1.6&nbsp;&nbsp;</span>Common pitfalls</a></span></li><li><span><a href="#Recap" data-toc-modified-id="Recap-2.1.7"><span class="toc-item-num">2.1.7&nbsp;&nbsp;</span>Recap</a></span></li></ul></li><li><span><a href="#Optuna-tutorial-2" data-toc-modified-id="Optuna-tutorial-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Optuna tutorial 2</a></span></li></ul></li><li><span><a href="#Hands-on" data-toc-modified-id="Hands-on-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hands-on</a></span></li></ul></div>

# Introduction

One of the most critical tasks in Deep Reinforcement Learning is to find a good set of training hyperparameters.



<img align="right" src="https://raw.githubusercontent.com/optuna/optuna/master/docs/image/optuna-logo.png" style=" width:200px; padding: 10px 20px;" title="Optuna Loog">

[Optuna]() is a library that helps you to automate the search. In this Unit, we’ll study <b>a little bit of the theory behind automatic hyperparameter tuning.</b> We’ll first try to optimize the parameters of the DQN studied in the last unit manually. We’ll then <b>learn how to automate the search using Optuna.</b>

# Optuna Tutorial

The content below comes from [Antonin’s Raffin ICRA 2022 presentations](https://araffin.github.io/tools-for-robotic-rl-icra2022/), he’s one of the founders of Stable-Baselines and RL-Baselines3-Zoo.

## The theory behind Hyperparameter tuning


[![Automatic Hyperparameter Optimization @ ICRA 22 | Tools for Robotic RL 6/8](http://img.youtube.com/vi/AidFTOdGNFQ/0.jpg)](http://www.youtube.com/watch?v=AidFTOdGNFQ "Automatic Hyperparameter Optimization @ ICRA 22 | Tools for Robotic RL 6/8")
<span style="font-size:small"><center><i>Automatic Hyperparameter Optimization @ ICRA 22 | Tools for Robotic RL 6/8.</i></center></span>

### Notes from Video


<b>Motivation</b>
- Fair comaparison with baselines 
    - While doing research, and we want to compare to baseline, usually, we tune our own methods a lot. But, to be fair, we also need to tune other baseline. 
- Automatic tuning (no more grad student descent)
    - Lot of time spent wathching different metrics and trying different configs
- Improve performance/training time
    - Reach given performance in given amount of time (e.g., 15 mins)

<b>Outline</b>

1. <b>Hyperparameter Optimization: "b vs B/n" tradeoff</b>
    - Trade-off between No. of configs vs. Budget<br><br>
    
2. <b>Samplers</b>
    - Different algos <br><br>
3. <b>Schedulers</b>
    - Different algos <br><br>
4. <b>In Practice (Optuna)</b>

### "N vs B/N" Tradeoff

<img src="attachments/n-vs-bn-tradeoff.png" style="width:700px;" title="N vs B/N tradeoff">

Reduce budget (or remove) the less promising trials, and give more budget (allocating more resources) to more promising trials as the time progress.


Two main components in hyperparameter optimization:
1. <b>Sampler</b> (search algo): Decides which config we need to try
2. <b>Pruner</b> (scheduler): Decides when to stop the trial

### Samplers (Search algo)

<img src="attachments/performance-landscape.png" style="width:700px;" title="Performance Landscape">

We don't have this Performance Landscape in advance. 

<img src="attachments/grid-search.png" style="width:800px;" title="Grid Search">

What we have been doign so far?
- Discretizing the parameter search space, but this can be quite tricky to reach optimal hyperparams, beacuse
    - <span style="color:red">we do not know how to discretize the search space in advance</span>
    - <span style="color:red">we end up allocating same budget for both important and unimportant paramater</span>
    - <span style="color:red">it scales very poorly (the search space explodes) as the n_params increases</span>

<img src="attachments/grid-search-vs-random-search.png" style="width:800px;" title="Grid Search vs. Random Search">

In the world of hyper parameter optimization, grid search is not used as a baseline anymore. Instead, a more complicated <b>random search</b> is used. Besides, random search is not affected by discretization.

Can we do better than random search? Yes, we can do 
- <b>Bayesian Optimization</b> or 
    - Gaussian Process
    - Tree of Parzen Estimators
- <b>Black Box Optimization</b>
    - Evolution Strategies
    - Practice Swarm Optimization


<img src="attachments/bayesian-optimization.png" style="width:800px;" title="Bayesian optimization">

Learn a Surrogate model. It's a model that given the a set of parameter, it will estimate the performance (black line in the fig above) we will get. This model come with some uncertainty (blue area in the fig above).

How it works?
- What we want to achieve is maximum performance (the black dotted line) which we don't know 
- Starts with a random config that we sample (a.k.a an observation as show in fig) and the uncertainty will go smaller near the observation
- Assemble the points that is the most promising one (by iteratively taking the surrogate model outputs and uncertainty into the acquistion function thaht tells where we should sample to may be get the best results)


### Schedulers (Prunner)

Few popular scheduler algos are
- Median pruner
- Successive halving

<b>Median Pruner</b>
- Simple heuristics
- Used in [Google Vizier](https://github.com/google/vizier)
- Prune if the trial's intermediate result is worse than median of intermediate results of previous trials at the same step.

<b>Successive Halving</b>

<img src="attachments/successive-halving.png" style="width:800px;" title="Successive halving">

### Optuna demo - HP Tuning Steps

- Define the search space
    - Params we are going to optimise and their boundaries
- Define the objective function
    - What we want to optimise? In RL, we optimise for preformance. But, we may optimise for energy efficiency, fast training, etc.
- Choose a sampler and pruner
- Get a coffee/ nap

#### <b>Search space</b>
- In optuna, there are 3 main types
    1. Categorical variables 
    2. Integers
    3. Floats

<img src="attachments/search-space-example.png" style="width:650px;" title="Search space example">

#### <b>RL Objective Function</b>
- Two parts
    1. Evaluation part
        - Evaluating periodically the trial and sending the report to Optune. Optuna tells whether we should prune or continue training based on the eval performance
    2. ...
        - Defining the model, sampling the hyperparameter, training the model, and sending back to optuna to find the performance. Steps involved are
        - Sample hyperparameters using `sample_ppo_params`
        - Create/Instantiate the RL model/algo using `PPO(...)`
        - Create/Instantiate the callback that will periodically evaluate and report the trial performance using `...`
        - Train the model using `model.learn(...)`
        - 

<img src="attachments/rl-objective-function-1-of-2.png" style="width:650px;" title="RL objective function (1/2)">
<img src="attachments/rl-objective-function-2-of-2.png" style="width:700px;" title="RL objective function (2/2)">

#### <b>Choose sampler, pruner and launch the study</b>

- Select a sampler, 
    - e.g., Bayesian optimization using `TPESampler(...)`. 
- Select a pruner
    - e.g., `MedianPruner(...)`
- Create the optuna study
    - e.g., We want to maximise the cumulative reward, so `optuna.create(..., direction="maximize")`
- Run in parallel
- Get the best result (trial with best type of parameter)

<img src="attachments/choose-sampler-pruner-launch-study.png" style="width:700px;" title="Choose sampler, pruner and launch the study!">

### Common pitfalls

- HP optimization not needed (train longer first)
- Noisy evaluation: Multiple eval 
    - Different random seed will provide different results
- Search space too small/wide
- Slow optimization: Smaller budget
- Training not stable: Manual tweaks
    - Reducing the learning rate, or augmenting the replay buffer size

### Recap

- Use automatic tuning when possible/needed
- Automatic tuning = sampler + pruner + objective function
- Do not use grid search
- Common pitfalls

## Optuna tutorial 2

[![Hyperparameter Tuning with Optuna Notebook @ ICRA 22 | Tools for Robotic RL 7/8
](http://img.youtube.com/vi/ihP7E76KGOI/0.jpg)](http://www.youtube.com/watch?v=ihP7E76KGOI "Hyperparameter Tuning with Optuna Notebook @ ICRA 22 | Tools for Robotic RL 7/8")

- My [source code](https://colab.research.google.com/drive/1dKECP2z6Qqj2Ci3iP0n6LOFkEH7XhO6c?usp=sharing) in Google Colab stored in Google Drive for building the model.

# Hands-on

Now that you’ve learned to use Optuna, here are some ideas to apply what you’ve learned:

1️⃣ <b>Beat your LunarLander-v2 agent results</b>, by using Optuna to find a better set of hyperparameters. You can also try with another environment, such as MountainCar-v0 and CartPole-v1.

2️⃣ <b>Beat your SpaceInvaders agent results.</b>

By doing this, you’ll see how valuable and powerful Optuna can be in training better agents.

Have fun!